In [8]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1490],
        [-0.2008]], grad_fn=<AddmmBackward0>)

In [9]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1967, -0.1217,  0.0233, -0.0832, -0.0577, -0.0611, -0.2309, -0.3459]])), ('bias', tensor([-0.0686]))])


In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0686], requires_grad=True)
tensor([-0.0686])


In [11]:
net[2].weight.grad == None

True

In [12]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [13]:
net.state_dict()['2.bias'].data

tensor([-0.0686])